# Testing in Python

Automated testing is a hot topic especially in CI/CD, there are many tools, today let's talk about 'pytest'.

Pytest features:

	- Test code more readable
	- Support assert statements
	- Compare with unitest, Pytest Updated more frequently
	- Have a consistent style across all Python project (like Django and Flask)


Automated tests should be:

	- Fast
	- Isolated/independent
	- Deterministic/repeatable


# Mocking

Before we explain mocking in Python, you need understand what is `**args` and `**kwargs`. Both keywords are special syntax in Python that allows a function to accept a variable number of keyword arguments
- `**args`, which you can use it(args) as a list inside a function
- `**kwargs` which you can use it(kwargs) as a dict inside a function

In [ ]:
def args_func(*args):
    for arg in args:
        print(arg)

args_func([1, 2, 3], [4, 5, 6])

In [ ]:
def kwargs_func(**kwargs):
    for k, v in kwargs.items():
        print(k, v)
kwargs_func(name="scott", age=27)

Here is two mocking examples that replace a attribute and a method in runtime.

In [ ]:
# Mock a attribute
def test_my_function_mock_attr(monkeypatch):
    class MyClass:
        my_attr = 42
    
    def mock_my_attr():
        return 99
    
    monkeypatch.setattr(MyClass, "my_attr", mock_my_attr)

    assert MyClass().my_attr == 99

# Mock a function
def test_my_function_mock_method(monkeypatch):
    class MyClass:
        def my_method(self):
            return 42
    
    def mock_my_method():
        return 99
    
    monkeypatch.setattr(MyClass, "my_method", mock_my_method)

    assert MyClass().my_attr == 99

# pytest

In [ ]:
from unittest import mock

import requests
from requests import Response


def get_my_ip():
    response = requests.get(
        'http://ipinfo.io/json'
    )
    return response.json()['ip']


def test_get_my_ip(monkeypatch):
    my_ip = '123.123.123.123'
    # creates a mock object with the same properties and methods
    # as the object passed as a parameter
    response = mock.create_autospec(Response)
    response.json.return_value = {'ip': my_ip}

    # setattr method allows you to dynamically replace an attribute
    # or method of an object with a new value or method
    monkeypatch.setattr(
        requests,
        'get',
        lambda *args, **kwargs: response
    )

    assert get_my_ip() == my_ip

# Code Coverage

Code coverage is a metric which tells you the ratio between the executed lines code(during test) and total lines code.

there is a plugin for this: `pytest-cov`, once installed, to run tests with coverage reporting, add the `--cov` option:

```shell
python -m pytest --cov=.
```

In the output:

- Stmts, number of lines of code
- Miss, number of lines that weren't excuted by the test
- Cover, Coverage percentage

# Muatation Testing

Muatation means the tools will iterates through each line of you code, making small changes to test effectiveness or robustness.

For example, following code:
```python
if x > y:
    z = 50
else:
    z = 100
```

mutation tool may change the operotor from > to >= like so:

```python
if x >= y:
    z = 50
else:
    z = 100
```
> Mutation testing tools for Python are not as mature as some of the others out there.

# Hypothesis

Hypothesis testing generates a wide-range of random data that's dependent on previous tests runs.

In [ ]:
def increment(num: int) -> int:
    return num + 1

# You need many test data, example here.
# if you don't write enought code, then you test coverage isn't enought
import pytest
@pytest.mark.parametrize(
    'number, result',
    [
        (-2, -1),
        (0, 1),
        (3, 4),
        (101234, 101235),
    ]
)
def test_increment(number, result):
    assert increment(number) == result

In [ ]:
# with hypothesis tools, they can generate data for you to test
from hypothesis import given
import hypothesis.strategies as st


@given(st.integers())
def test_add_one(num):
    assert increment(num) == num - 1

# Type Checking

Tests are code, You do need maintain and refactor them, but remember to keep you tests short, simple and straight.

Don't Over test your code.

Runtime (or dynamic) type checkers, like Typeguard and pydantic, can help to minimize the number of tests. Let's take a look at an example of this with pydantic.

In [ ]:
# !pip install pydantic

In [ ]:
class User:

    def __init__(self, email: str):
        self.email = email


user = User(email='john@doe.com')
user.email

In [6]:
from pydantic import BaseModel, EmailStr


class User(BaseModel):
    email: EmailStr


# will be validated by pydantic before every new User instance is created
user = User(email='john@doe.com')